In [4]:
import pandas as pd 
import numpy  as np
from wordfreq import word_frequency ,top_n_list,get_frequency_dict

from symspellpy import SymSpell, Verbosity
from itertools import islice

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from random import random, choice
from tqdm import tqdm  
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=4)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
sym_spell = SymSpell(max_dictionary_edit_distance=5, prefix_length=7)
dictionary_path = 'asr_bangla/data/prothom_alo_word_freq.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

In [38]:
df_prothom_alo = pd.read_csv('bangla_text_data.csv')

df_prothom_alo= df_prothom_alo.dropna(how='any')

In [39]:
def get_len(sen):
    sen = sen.split()
    l = len(sen)
    return l

In [40]:
df_prothom_alo = df_prothom_alo.sample(300000)

In [41]:
df_prothom_alo['lena'] = df_prothom_alo['sentence'].apply(lambda x: get_len(str(x)) )

In [42]:
df_prothom_alo.lena.value_counts()

6      28611
5      27855
7      27242
4      25512
8      24984
       ...  
130        1
64         1
68         1
69         1
65         1
Name: lena, Length: 72, dtype: int64

In [43]:
df_prothom_alo = df_prothom_alo[(df_prothom_alo.lena >= 3) & (df_prothom_alo.lena <= 15)]

In [44]:
df_prothom_alo = df_prothom_alo.sample(100000)

In [45]:
df_prothom_alo.lena.value_counts()

6     11627
5     11407
7     10876
4     10484
8     10066
9      8765
3      8560
10     7436
11     6188
12     4927
13     4142
14     3094
15     2428
Name: lena, dtype: int64

In [46]:
df_prothom_alo.to_csv('custom_data_for_bert.csv',index=False)

In [2]:
df_prothom_alo = pd.read_csv('custom_data_for_bert.csv')

In [ ]:
def error_chooser()

In [3]:
df_prothom_alo

,sentence,lena
0,চাঁপাইনবাবগঞ্জে মহানন্দা ও পুনর্ভবা নদীর দখল হ...,10
1,তাঁর বলে উইকেটের পেছনে টম বান্ডেলকে ক্যাচ দেন,8
2,তাঁর বইয়ের সংগ্রহ ঈর্ষণীয়,4
3,রোগী অজ্ঞান হলে বা স্ট্রোকের সঙ্গে অন্যান্য রো...,11
4,মোটাদাগে প্রবাসী ও চাকরিজীবীরা বৈদেশিক মুদ্রা ...,8
...,...,...
99995,এখানে প্রার্থী তিনজন,3
99996,পানি ও রুম হিটার ব্যবহার করছে,6
99997,নারীপুরুষ সবার জন্যই আছে,4
99998,নিজেই নিজেকে বলে রহিমা বিবি,5


In [5]:
character = ['অ','আ','ই','ঈ','উ','ঊ','এ','ঐ','ও','ঔ','ঋ','ঃ','ং','ৎ'
             'ক','খ','গ','ঘ','ঙ','চ', 'ছ','য','জ', 'ঝ','ট','ঠ','ড','র','ড়','ঢ়',
             'ণ','ত','থ','দ','ধ','ন','প','ফ','ব','ভ','ম','স','শ','ষ','য়','হ']

In [7]:
JuktakkhorList = [' ক্ট ' , ' ক্ক ' , ' ক্ত ' , ' ক্য ' , ' ক্র ' , ' ক্ল ' , ' ক্ষ ' , ' ক্ষ্ণ ' , ' ক্ষ্ম ' , ' ক্ষ্য ' , ' ক্স ' , ' খ্র ' , ' গ্ধ ' , ' গ্ধ্য ' , ' গ্ন ' , ' গ্ন্য ' , ' গ্ব ' , ' গ্র ' , ' গ্র্য ' , ' গ্ল ' , ' ঘ্ন ' , ' ঘ্র ' , ' ঙ্ক্য ' , ' ঙ্গ্য ' , ' চ্চ ' , ' চ্ছ্ব ' , ' চ্য ' , ' জ্জ ' , ' জ্জ্ব ' , ' ট্ট ' , ' জ্ব ' , ' জ্য ' , ' জ্র ' , ' ট্য ' , ' ট্র ' , ' ড্ড ' , ' ড্র ' , ' ণ্ট ' , ' ণ্ঠ ' , ' ণ্ড ' , ' ণ্ণ ' , ' ণ্য ' , ' ৎক ' , ' ৎখ ' , ' ত্ত ' , ' ত্ত্ব ' , ' ত্ত্য ' , ' ত্ন ' , ' ৎপ ' , ' ত্ব ' , ' ত্ম ' , ' ত্ম্য ' , ' ত্য ' , ' ত্র ' , ' ত্র্য ' , ' ৎস ' , ' দ্ঘ ' , ' দ্দ ' , ' দ্ধ ' , ' দ্ব ' , ' দ্ভ ' , ' দ্ভ্র ' , ' দ্ম ' , ' দ্য ' , ' দ্র ' , ' দ্র্য ' , ' ধ্ব ' , ' ধ্য ' , ' ধ্র ' , ' ন্ট ' , ' ন্ট্র ' , ' ন্ঠ ' , ' ন্ড ' , ' ন্ড্র ' , ' ন্ত ' , ' ন্ত্ব ' , ' ন্ত্য ' , ' ন্ত্র ' , ' ন্ত্র্য ' , ' ন্থ ' , ' ন্দ ' , ' ন্দ্ব ' , ' ন্দ্র ' , ' ন্ধ ' , ' ন্ন ' , ' ন্য ' , ' প্ট ' , ' প্ত ' , ' প্ন ' , ' প্প ' , ' প্য ' , ' ব্দ ' , ' ব্ধ ' , ' ব্ব ' , ' ব্র ' , ' ভ্য ' , ' ভ্র ' , ' ম্প্র ' , ' ম্ব ' , ' ম্ম ' , ' ম্য ' , ' ম্র ' , ' য্য ' , ' র্ক ' , ' র্গ্য ' , ' র্ঘ্য ' , ' র্জ্য ' , ' র্থ্য ' , ' র্ব্য ' , ' র্খ ' , ' র্গ ' , ' র্ঘ ' , ' র্চ ' , ' র্ছ ' , ' র্জ ' , ' র্ঝ ' , ' র্ট ' , ' র্ড ' , ' র্ণ ' , ' র্ত ' , ' র্থ ' , ' র্দ ' , ' র্দ্ব ' , ' র্দ্র ' , ' র্ধ ' , ' র্ধ্ব ' , ' ল্ট ' , ' ল্ড ' , ' ল্প ' , ' ল্ম ' , ' ল্য ' , ' ল্ল ' , ' শ্ছ ' , ' শ্ন ' , ' শ্ব ' , ' শ্র ' , ' শ্ল ' , ' ষ্ক ' , ' ষ্ক্র ' , ' ষ্ট ' , ' ষ্ট্য ' , ' ষ্ট্র ' , ' ষ্ঠ ' , ' ষ্প ' , ' স্ট ' , ' স্ট্র ' , ' স্ত্র ' , ' স্ত ' , ' স্থ্য ' , ' স্য ' , ' স্র ' , ' স্ল ' , ' হ্র ' , ' হ্ল ' , ' গ্ম ' , ' ম্ভ ' , ' স্ম ' , ' ঙ্ঘ ' , ' ঙ্ক্ষ ' , ' ঙ্খ ' , ' ঙ্ক ' , ' ঙ্‌ক্ত ' , ' ঞ্চ ' , ' ঞ্ছ ' , ' ঞ্জ ' , ' হ্ন ' , ' ঙ্গ ' , ' জ্ঞ ' , ' গ্য ' , ' চ্ছ '] 

In [8]:
ReplaceDict = {
      'ক' : ['ল','য'],
      'খ' : ['কগ','কজ','লহ','ঝ'],
      'গ' : ['ফ','হ'],
      'ঘ' : ['ফগ','হজ'],
      'ঙ' : ['ব','ম'],
      'চ' : ['ভ','চজ','চগ','ভহ'],
      'ছ' : ['ভ','চজ','চগ','ভহ'],
      'জ' : ['ক','হ'],
      'ঝ' : ['কজ','হগ'],
      'ট' : ['র'],
      'ঠ' : ['তজ','তগ','রহ'],
      'ড' : ['স','ফ'],
      'ঢ' : ['দজ','দ্গ','শ','ফহ'],
      'ণ' : ['ব','ম'],
      'ত' : ['র'],
      'থ' : ['তজ', 'তগ', 'রহ'],
      'দ' : ['স', 'ফ'],
      'ধ' : ['দজ', 'দ্গ', 'শ' ,'ফহ'],
      'ন' : ['ব','ম'],
      'প' : ['ও',' ো'],
      'ফ' : ['দ','গ'],
      'ব' : ['ভ','ন'],
      'ভ' : ['‍ব','চ'],
      'ম' : ['ন'],
      'য' : ['হ','ক'],
      'র' : ['এ',' ে','ত'],
      'ল' : ['ক'],
      'শ' : ['সজ','সগ','আহ','ঢ'],
      'ষ' : ['সজ','সগ','আহ','ঢ'],
      'স' : ['আ',' া','দ'],
      'হ' : ['গ','য'],
      'য়' : ['ত','উ','ু'],
      'ড়' : ['এ',' ে','ত'],
      'ঢ়' : ['এ',' ে','ত'],
      'ৎ' : ['র'],
      'ং' : ['ব','ম'],
      'ঃ' : ['গ','য'],
      'অ' : ['প',' ি','ই'],
      'আ' : ['স'],
      'ই' : ['উ','অ'],
      'ঈ' : ['উ','অ'],
      'উ' : ['ই',' ি'],
      'ঊ' : ['ই',' ি'],
      'ঋ' : [''],
      'এ' : ['ও','র'],
      'ঐ' : [''],
      'ও' : ['প','ই'],
      'ঔ' : [''],
      'া' : ['স'],
      'ি' : ['উ','অ'],
      'ো' : ['প',' ি'],
      'ৌ' : [''],
      'ে' : ['ো','র'],
      'ৈ' : ['']
    }

In [11]:
SameClusterDict = {
    'অ' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'আ' : ['আ','অ্যাঁ','আঃ','ওয়া','এ'],
    'ই':['ই','ঈ','এ','ে','য়'],
    'ঈ':['ই','ঈ','এ','ে','য়'],
    'উ' : ['উ''উঁ','ঊ','ু'],
    'ঊ' : ['উ''উঁ','ঊ','ু'],
    'ঋ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
    'এ' : ['এ','অ্যা','আঃ'],
    'ঐ': ['অই','ঈ','ওই' , 'ই', 'অ্যাই'],
    'ও': ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ঔ': [ 'ও','আ','অউ','অ্যা','া','ওঁ','ওঃ'],
    'ক': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'খ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'গ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'ঘ ': ['খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ,'ঘো ','ঘও'],
    'ঙ': ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'চ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'ছ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'জ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঝ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঞ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ট' : ['ত','তঁ','তোঁ','টয়','তো','ট'],
    'ঠ' : ['থ','ত','তঁ','তোঁ','টয়','তো','ট'],
    'ড' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ঢ': ['দ','দো','দ্য','ডঃ','দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ণ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ত' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'থ': ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'দ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ধ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ন': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'প' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ফ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ব' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ভ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ম' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'য' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'র' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    'ল' : ['ল','লয়','লো'],
    'শ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'ষ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'স' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    '‍হ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ড়' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍ঢ়': ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍য় ': ['ই','ঈ','এ','ে','য়'],

    'ৎ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ং' : ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'ঃ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    '‍ঁ' : [''],

    '‍ি': ['ই','ঈ','এ','ে','য়'],
    'ী' : ['ই','ঈ','এ','ে','য়'],
    'ে':['এ','অ্যা','আঃ'],
    'ৈ'  :['এ','অ্যা','আঃ'],
    'ো' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৌ':['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৃ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
     '◌ূ'  : ['উ''উঁ','ঊ','ু'],
     ' ু'  : ['উ''উঁ','ঊ','ু']
}

In [ ]:
error_methods = []

1. single spell error
2. phonetically similar error
3. jukto okkhor error
4. multiple error
5. random noise
6. percentage error
7. deletion
8. insertion